In [ ]:
#!nvidia-smi

In [ ]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=0

In [1]:
import os
import numpy as np
from scipy.misc import imread, imresize
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import random

In [2]:
mnist = input_data.read_data_sets("../data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

Extracting ../data/train-images-idx3-ubyte.gz
Extracting ../data/train-labels-idx1-ubyte.gz
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz


In [3]:
# hyper parameters
learning_rate = 0.001
training_epochs = 70
batch_size = 64
n_class = 10
seed = 777
tf.set_random_seed(seed)
cur_dir = os.getcwd()

In [4]:
def preproc(x):
    # x = x*2 - 1.0
    # per-example mean subtraction (http://ufldl.stanford.edu/wiki/index.php/Data_Preprocessing)
    mean = tf.reduce_mean(x, axis=1, keep_dims=True)
    return x - mean

In [5]:
def conv_bn_activ_dropout(name, x, n_filters, kernel_size, strides, dropout_rate, training, seed, 
                          padding='SAME', activ_fn=tf.nn.relu):
    with tf.variable_scope(name):
        net = tf.layers.conv2d(x, n_filters, kernel_size, strides=strides, padding=padding, use_bias=False,
                              kernel_initializer=tf.contrib.layers.xavier_initializer(seed=seed))
        net = tf.layers.batch_normalization(net, training=training)
        net = activ_fn(net)
        if dropout_rate > 0.0:            
            net = tf.layers.dropout(net, rate=dropout_rate, training=training, seed=seed)
    return net

In [6]:
def build_vgg(X_img, dropout_rate, seed):
    net = X_img
    n_filters = 64
    for i in range(3):
        net = conv_bn_activ_dropout(name="3x3conv{}-{}".format(i+1, 1), x=net, n_filters=n_filters, 
                                   kernel_size=[3,3], strides=1, dropout_rate=dropout_rate, 
                                   training=is_train, seed=seed)
        net = conv_bn_activ_dropout(name="3x3conv{}-{}".format(i+1, 2), x=net, n_filters=n_filters, 
                                   kernel_size=[3,3], strides=1, dropout_rate=dropout_rate, 
                                   training=is_train, seed=seed)
        if i == 2:
            net = conv_bn_activ_dropout(name="1x1conv", x=net, n_filters=n_filters,
                                       kernel_size=[1,1], strides=1, dropout_rate=dropout_rate,
                                       training=is_train, seed=seed)
        n_filters *= 2
        net = conv_bn_activ_dropout(name="5x5stridepool{}".format(i+1), x=net, n_filters=n_filters,
                                   kernel_size=[5,5], strides=2, dropout_rate=dropout_rate,
                                   training=is_train, seed=seed)
    
    net = tf.contrib.layers.flatten(net)
    logits = tf.layers.dense(net, 10, kernel_initializer=tf.contrib.layers.xavier_initializer(seed=seed),
                            name="logits")
    
    return logits

In [7]:
X = tf.placeholder(tf.float32, [None, 784], name="X")
Y = tf.placeholder(tf.float32, [None, n_class], name = "Y")
is_train = tf.placeholder(tf.bool, name="is_train")

In [8]:
X_pre = preproc(X)
X_img = tf.reshape(X_pre, [-1, 28, 28, 1], name="X_img")

In [9]:
logits = build_vgg(X_img, dropout_rate=0.3, seed=seed)

In [10]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="loss")
global_step = tf.Variable(0, trainable=False)
learningRate = tf.train.exponential_decay(learning_rate=learning_rate,
                                          global_step= global_step,
                                          decay_steps=5000,
                                          decay_rate= 0.1,
                                          staircase=True)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(loss, name="optimizer") 

In [11]:
pred = tf.argmax(logits, axis=1, name="prediction")
prob = tf.nn.softmax(logits, name="softmax")
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(Y, axis=1)), tf.float32), name="accuracy")

In [12]:
train_var = [X, Y, is_train, logits, pred, prob, accuracy]
tf.add_to_collection('train_var', train_var[0])
tf.add_to_collection('train_var', train_var[1])
tf.add_to_collection('train_var', train_var[2])
tf.add_to_collection('train_var', train_var[3])
tf.add_to_collection('train_var', train_var[4])
tf.add_to_collection('train_var', train_var[5])
tf.add_to_collection('train_var', train_var[6])
saver = tf.train.Saver()
##saver.export_meta_graph(os.path.join(cur_dir, 'checkpoints', 'mnist_ckpt.meta'), collection_list=['train_var'])

In [13]:
# initialize
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(tf.global_variables_initializer())

In [14]:
# train my model
print('Learning started. It takes sometime.')
max_test_acc = 0.
for epoch in range(training_epochs):
    avg_cost = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    
    total_batch = int(mnist.train.num_examples / batch_size)
    total_batch_test = int(mnist.test.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        feed_dict = {X: batch_xs, Y: batch_ys, is_train:True}
        acc, c, _ = sess.run([accuracy, loss, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        avg_train_acc += acc / total_batch
        
    for i in range(total_batch_test):
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)        
        #batch_xs = batch_xs.reshape(-1, time_steps, element_size)
        feed_dict = {X: batch_xs, Y: batch_ys, is_train:False}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        avg_test_acc += acc / total_batch_test
    if avg_test_acc > max_test_acc:
        max_test_acc = avg_test_acc
        print('saving a graph and weights => accuracy : {}'.format(max_test_acc))
        saver.save(sess, os.path.join(cur_dir, 'checkpoints', 'mnist_vggsmall128.ckpt'))

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 
          'train accuracy = ', '{:.5f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.5f}'.format(avg_test_acc))


print('Learning Finished!')

Learning started. It takes sometime.
saving a graph and weights => accuracy : 0.9069511217948717
Epoch: 0001 cost = 0.379516512 train accuracy =  0.90241 test accuracy =  0.90695
saving a graph and weights => accuracy : 0.9660456730769239
Epoch: 0002 cost = 0.082020850 train accuracy =  0.97644 test accuracy =  0.96605
saving a graph and weights => accuracy : 0.990584935897435
Epoch: 0003 cost = 0.060707774 train accuracy =  0.98243 test accuracy =  0.99058
Epoch: 0004 cost = 0.058587541 train accuracy =  0.98312 test accuracy =  0.98478
Epoch: 0005 cost = 0.048384012 train accuracy =  0.98570 test accuracy =  0.97877
saving a graph and weights => accuracy : 0.9917868589743581
Epoch: 0006 cost = 0.043466364 train accuracy =  0.98742 test accuracy =  0.99179
Epoch: 0007 cost = 0.036616619 train accuracy =  0.98967 test accuracy =  0.98988
saving a graph and weights => accuracy : 0.9929887820512809
Epoch: 0008 cost = 0.032828367 train accuracy =  0.99077 test accuracy =  0.99299
Epoch: 0

In [15]:
#saver.save(sess, os.path.join(cur_dir, 'checkpoints', 'mnist_save.ckpt'))

In [16]:
def evaluate(X_sample, y_sample, batch_size=100):
    """Run a minibatch accuracy op"""

    N = X_sample.shape[0]
    correct_sample = 0

    for i in range(0, N, batch_size):
        X_batch = X_sample[i: i + batch_size]
        y_batch = y_sample[i: i + batch_size]
        N_batch = X_batch.shape[0]

        feed = {
            X: X_batch,
            Y: y_batch,
            is_train: False
        }

        correct_sample += sess.run(accuracy, feed_dict=feed) * N_batch

    return correct_sample / N

print("\nAccuracy Evaluates")
print("-------------------------------")
print('Train Accuracy:', '{:.5f}'.format(evaluate(mnist.train.images, mnist.train.labels)))
print('Test Accuracy:', '{:.5f}'.format(evaluate(mnist.test.images, mnist.test.labels)))


Accuracy Evaluates
-------------------------------
Train Accuracy: 0.99989
Test Accuracy: 0.99560
